In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import glob
import os
import numpy as np
import glob
import json
import pickle
import pandas as pd
from lbt_recipes.settings import RecipeSettings
from lbt_recipes.recipe import Recipe
from honeybee.model import Model
from ladybug import wea, epw
import shutil
from pathlib import Path
import sys
import project_func as pf
module_path = r"C:\Users\Justin\Documents\GitHub\IPV_Workbench"
sys.path.insert(0, module_path)
# from ipv_workbench.solver import simulations as ipv_sim
from ipv_workbench.utilities import utils, time_utils, circuits
# from ipv_workbench.solver import calculations as ipv_calc
# from ipv_workbench.translators import panelizer
# from ipv_workbench.translators import results_writers as ipv_results
# from ipv_workbench.devices import devices
# from ipv_workbench.translators import mapping_irradiance as ipv_irrad
# from ipv_workbench.solver import simulations, simple_power_models


project_root = r"C:\Users\Justin\Documents\Github\cmip6_and_buildings"

# Fix EPWs

In [27]:
from epw import epw  # https://github.com/building-energy/epw



def fix_epw(base_epw):
    a = epw()
    a.read(base_epw)
    new_epw = a.dataframe
    new_df = utils.tmy_to_dataframe(base_epw)
    new_epw['Direct Normal Radiation'] = new_df['dirnorrad_Whm2'].fillna(0)
    return a


def out_epw(epw_object,file_path):
    epw_object.write(file_path)




In [21]:
cz_list = pf.get_cz_list()
years = pf.get_years()
vintages = pf.get_vintages()
years = pf.get_years()
scenarios = pf.get_scenarios()

In [30]:
weather_dir = os.path.join(project_root, 'weather_files', 'morphed')

for cz in cz_list:
    for scenario in scenarios:
        epw_dir = os.path.join(weather_dir, cz, scenario, 'EPWs')
        epw_files = glob.glob(os.path.join(epw_dir,"*.epw"))
        for fpath in epw_files:
            new_epw = fix_epw(fpath)
            out_epw(new_epw, fpath)
#     for year in years:
#         for scenario in scenarios:
#             for cz in cz_list:
                    

In [29]:
test_path = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\weather_files\morphed\1a\ssp126\EPWs\ssp126_50_2050-2080.epw"

utils.tmy_to_dataframe(test_path)['dirnorrad_Whm2']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8755    0.0
8756    0.0
8757    0.0
8758    0.0
8759    0.0
Name: dirnorrad_Whm2, Length: 8760, dtype: float64

# fix DNI in historical

# copy historical

In [33]:
for vintage in vintages:
    vintage_dir = os.path.join(project_root, 'eplus_simulations', vintage[0])
    for cz in cz_list[1:]:
        old_results = os.path.join(vintage_dir, 'results', 'pv_results',cz)
        src_dir = os.path.join(old_results,'historical_2020')
        dst_dir = os.path.join(vintage_dir,'results','irrad_results',cz,'historical_2020')
        shutil.move(src_dir, dst_dir)

# Run Loop

In [3]:
def load_model(model_path):
    with open(model_path,'r') as fp:
        sim_model_dict = json.load(fp)
    return sim_model_dict


def create_results_dir(project_dir, scenario, year, vintage, climate_zone, building_type):

    # if year==2020:
    #     pass
    # else:
    #     year =int((int( year.split("-")[-1]) - int(year.split("-")[0])) / 2 + int(year.split("-")[0]))

    scen_year = str(scenario) + "_" + str(year)

    main_dir = os.path.join(project_dir, 'eplus_simulations',vintage[0],'results')
    pv_dir = os.path.join(main_dir, 'irrad_results', climate_zone, scen_year)
    results_dir = os.path.join(pv_dir, building_type)
    irradiance_dir = os.path.join(results_dir, 'irrad')
    if os.path.exists(irradiance_dir):
        pass
    else:
        os.makedirs(irradiance_dir)
    return irradiance_dir


def run_simulation(model,wea_path,folder,rad_par,workers_,run=True):
    start = time.time()

    # set the threshold
    # sensor_pts = len(model.properties.radiance.to_dict()['radiance']['sensor_grids'][0]['sensors'])
    # threshold = int(np.ceil(sensor_pts/workers_))
    threshold = 250
    
    # create the settings
    rec_settings = RecipeSettings(folder, workers=workers_)
    rec_settings.report_out = True

    # create the recipe and set the input arguments
    recipe = Recipe('annual-irradiance')
    recipe.input_value_by_name('model', model)
    recipe.input_value_by_name('wea', wea_path)
    recipe.input_value_by_name('north', 0)
    recipe.input_value_by_name('min-sensor-count', threshold)
    recipe.input_value_by_name('output-type', 'solar')
    recipe.input_value_by_name('radiance-parameters', rad_par)
    
    # run the recipe
    if run==True:
        recipe.run(settings=rec_settings, radiance_check=True, silent=True)
                
    end = time.time()
    total_runtime = round(end-start,3)
    print(f"Completed in {total_runtime} seconds")

    return total_runtime


def run_worfklow(project_root, workers, vintage, building_type, scenario, year, cz, rad_par, run_boolean=True):

    # set up model path
    model_path = os.path.join(project_root,'eplus_simulations',vintage[0],'results',"irrad_results",f"{building_type}{vintage[1]}",
                              'irrad',"irrad_model.hbjson")

    # load the honeybee model json as dict
    model_dict = load_model(model_path)

        
    rad_model = Model.from_dict(model_dict)

    # get wea
    if scenario=='historical':
        epw_file = glob.glob(os.path.join(project_root,"weather_files","morphed",f"{cz}", scenario, 
                                          'EPWS','baseline.epw'))[0]
    else:
        year_band = pf.get_year_bands(year)
        epw_file = glob.glob(os.path.join(project_root,"weather_files","morphed",f"{cz}",scenario,
                                          'EPWs',f'*50_{year_band}*.epw'))[0]
    print(epw_file)
    # check on results destination
    results_folder = create_results_dir(project_root, scenario, year, vintage, cz, building_type)
    
    # run
    if run_boolean==True:
        total_runtime = run_simulation(rad_model, epw_file, results_folder, rad_par, workers, run=run_boolean)
        print(f"------- {total_runtime}")

    # if run_boolean==True:
    #     ill_file_maybe = os.path.join(results_folder,"annual_irradiance","results","total",
    #                                     f"radiancegrid_{surface_name.replace('surface_','')}_{sensor_pt_count}sensors.ill")
    #     n_negative = check_negatives(ill_file_maybe)
    #     write_string = f"\n{time.ctime()},{condition},{surface_name},{sensor_pt_count},{total_runtime},{n_negative},{workers},{rad_par}"
    #     log_run(runtime_log,write_string)

In [4]:
cz_list = pf.get_cz_list()
years = pf.get_years()
vintages = pf.get_vintages()
years = pf.get_years()
scenarios = pf.get_scenarios()
rad_par = "-ab 3 -ad 15000 -as 2048 -c 1 -dc 0.5 -dp 256 -dr 1 -ds 0.25 -dt 0.25 -lr 6 -lw 6.67e-07 -ss 0.7 -st 0.5"

In [8]:
years

[2035, 2050, 2065, 2080]

In [6]:
for vintage in vintages[0:1]:
    for year in years[0:1]:
        for scenario in scenarios[0:1]:
            if scenario=='historical':
                if year==2020:
                    pass
                else:
                    continue
            for cz in cz_list:
                print(year, scenario, cz)
                for building_type in ['RefBldgMediumOffice','RefBldgMidriseApartment']:
                    run_worfklow(project_root, 22,
                                vintage, building_type,
                                scenario, year, cz, rad_par,
                                run_boolean=True)

2020 historical 1a
C:\Users\Justin\Documents\Github\cmip6_and_buildings\weather_files\morphed\1a\historical\EPWS\baseline.epw
b'2023-05-14 07:19:19 INFO: Started running CreateRadFolder...\r\n2023-05-14 07:19:19 INFO: Started running CreateDirectSky...\r\n2023-05-14 07:19:19 INFO: Started running CreateTotalSky...\r\n2023-05-14 07:19:20 INFO: Started running GenerateSunpath...\r\n2023-05-14 07:19:20 INFO: C:\\Program Files\\ladybug_tools\\python\\lib\\site-packages\\honeybee_radiance_folder\\folder.py:619: UserWarning: RefBldgMediumOffice sensor grid has no light-path. It will not be included in three or five phase studies.\r\n2023-05-14 07:19:20 INFO: \'or five phase studies.\' % grid[\'name\']\r\n2023-05-14 07:19:20 INFO: C:\\Users\\Justin\\AppData\\Local\\Temp\\tmp3xnbgw8c\r\n2023-05-14 07:19:20 INFO: ...finished running CreateRadFolder in 0:00:01\r\n2023-05-14 07:19:20 INFO: Started running CreateSkyDome...\r\n2023-05-14 07:19:20 INFO: [{"path": "sky.mtx", "full_path": "C:\\\\Users